<a href="https://colab.research.google.com/github/anatorres09/DE-MRI_Classificacio_CNN/blob/main/Copy_of_CNN_Classificacio_DE_MRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Seleccionar versió correcte tensorflow

In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# 2. Clonar projecte GitHub (accés ràpid DataSet)

In [2]:
!git clone https://github.com/anatorres09/DE-MRI_Classificacio_CNN.git

Cloning into 'DE-MRI_Classificacio_CNN'...
remote: Enumerating objects: 919, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 919 (delta 11), reused 57 (delta 11), pack-reused 862
Receiving objects: 100% (919/919), 166.34 MiB | 13.38 MiB/s, done.
Resolving deltas: 100% (56/56), done.
Updating files: 100% (153/153), done.


In [3]:
! ls

DE-MRI_Classificacio_CNN  sample_data


In [4]:
import tensorflow as tf

In [5]:
# Iniciar CNN
model = tf.keras.models.Sequential()

# 3. Afegir capa convolucional

In [6]:
# 32 detectors de característiques amb 3*3 dimensions, així que la capa convolucional esta composta per mapes de 32 característiques
# 128 per 128 dimensions amb imatges en color (3 canals) (tensorflow backend)
input_size = (128, 128)
model.add(tf.keras.layers.Convolution2D(32, 3, 3, input_shape = (*input_size, 3), activation = 'relu'))

# 4. Afegir capa de conjunt

In [7]:
# Reduir la mida dels mapes de característiques i reduir el nombre de nodes a la futura capa totalment connectada (reduir temps de complexitat, 
# menys càlcul intens sense perdre el rendiment). La dimensió 2 per 2 és l'opció recomanada
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))

# 5. Afegir segona capa convolucional amb ampliament

In [8]:
model.add(tf.keras.layers.Convolution2D(32, 3, 3, activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))

#6. Afegir capa d'aplanació

In [9]:
# aplaneu tots els mapes de característiques de la capa d'agrupació en un sol vector
model.add(tf.keras.layers.Flatten())

# 7. Afegir una capa totalment connectada

In [10]:
# making classic ann which compose of fully connected layers
# number of nodes in hidden layer (output_dim) (common practice is to take the power of 2)
model.add(tf.keras.layers.Dense(units = 64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# 8. Compilació del model

In [11]:
# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# 9. Ajustar la CNN a les imatges

In [13]:
# Llibreries
import os
import numpy as np
import nibabel as nib


# Directorio donde se encuentran las imágenes en formato NIfTI
nifti_dir = 'dataset/entrenament/normal'


# Obtener la lista de archivos en el directorio y ordenarla alfabéticamente
lista_archivos = sorted(os.listdir(nifti_dir))

#Especifica las dimensiones de una imagen
dimensiones = (128, 128) + (len(lista_archivos),)

# Crear un arreglo NumPy vacío para almacenar todas las imágenes
arreglo_imagenes = np.zeros(dimensiones, dtype=np.float16) #modificar el float si es necesario (32,64...)

# Cargar todas las imágenes en el arreglo
for i, archivo in enumerate(lista_archivos):
    imagen = nib.load(os.path.join(nifti_dir, archivo)).get_fdata()
    arreglo_imagenes[:, :, i] = imagen
arreglo_imagenes = np.transpose(arreglo_imagenes, (2, 0, 1))

# Guardar el arreglo de imágenes en formato NumPy
np.save('dataset/entrenament/normal/prueba.npy', arreglo_imagenes)


# image augmentation technique to enrich our dataset(training set) without adding more images so get good performance  results with little or no overfitting even with the small amount of images
# used from keras documentation (flow_from_directory method)

from keras.preprocessing.image import ImageDataGenerator
batch_size = 32
# image augmentation part
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

# create training set
# wanna get higher accuracy -> inccrease target_size
training_set = train_datagen.flow_from_directory('dataset/entrenament/normal/prueba.npy', 
                                                 target_size = input_size,
                                                 batch_size = batch_size,
                                                 class_mode = 'binary')

# create test set
# wanna get higher accuracy -> inccrease target_size
test_set = test_datagen.flow_from_directory('dataset/entrenament/normal/prueba.npy',
                                            target_size = input_size,
                                            batch_size = batch_size,
                                            class_mode = 'binary')

# fit the cnn model to the trainig set and testing it on the test set
model.fit(training_set,
          steps_per_epoch = 8000/batch_size,
          epochs = 35,
          validation_data = test_set,
          validation_steps = 2000/batch_size)

FileNotFoundError: ignored

# 10. Fer noves preddicions

In [ ]:
from keras.preprocessing import image

In [ ]:
test_image = image.load_img('C:/Users/anato/OneDrive/Escritorio/Entrenament/prueba.npy', target_size= input_size)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

In [ ]:
training_set.class_indices

In [ ]:
if result [0][0] == 1:
  prediction = 'patològic'
else:
  prediction = 'sa'

In [ ]:
prediction